# Manipulation Modelling & Execution

The discussed components (KOMO, BotOp, NLP_Solver, RRT) provide basic ingredients for manipulation planning and execution. This tutorial is about how to practically use these in typical manipulation settings.

The first focus is on *manipulation modelling*. While KOMO provides a very powerful abstract framework to define all kinds of constraints, here we discuss what are concrete useful constraints for typical actions, e.g., picking and placing a box, or capsule. The *ManipulationModelling* class is meant to translate between typical actions and the abstract KOMO specification of the corresponding constraints.

The second focus is on the whole pipeline. We follow a basic sense-plan-act pipeline (not yet a fully integrated reactive framework such as SecMPC). To be more detailed, we assume the following basic steps in each loop:
* Perception: Update the configuration model to be in sync with the real world - using perception.
* Discrete decisions (task planning): Decide on discrete next actions, such as which object to pick or place next.
* Waypoint planning: Model the manipulation constraints for the next few actions and solve them to get a plan for the next few waypoints.
* Path planning: Create a fine-grained path/trajectory between waypoints, sometimes justing quick interpolation & optimization, sometimes using full fledge path finding (bi-directional RRT).
* Execution: Sending the path to BotOp for running it on the real system.

We neglect perception and discrete decision making here.

## Manipulation Modelling

We start with discussing manipulation modelling for standard box/cylinder grasping and placing.

In [7]:
import robotic as ry
import numpy as np
import random
import time

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

A basic configuration with a box and cylinder:

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.06,.06,.005]) \
    .setColor([1,.5,0]) \
    .setContact(1)

C.addFrame("capsule") \
    .setShape(ry.ST.capsule, [.2,.02]) \
    .setPosition([.25,.1,1.]) \
    .setColor([1,.5,0]) \
    .setContact(1)

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

Look into the definition of *ManipulationModelling* class! You see that this class provides wrapper methods to setup a komo problem. The following demonstrate the methods provided to model box and cylinder grasping IK problems:

### Box centered top grasp
There are 6 possible orientation of an orthonormal centered box grasp. Have a look at the `grasp_top_box` method!

In [11]:
C.setJointState(qHome)
for orientation in ['xy', 'xz', 'yx', 'yz', 'zx', 'zy']: #loops over the 6 possible grasps
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    man.grasp_top_box(1., gripper, box, orientation)
    
    # solve it
    pose, ret = man.solve()
    
    # check feasibility and display
    if ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_top_box with orientation {orientation}\nret: {ret}')
    else:
        print(' -- infeasible')

### Box general grasp
We do not have to grasp a box in the center or orthonormally. We can only specify along which axis the fingers should press, and that they need to be inside a margin of the box sides.  Have a look at the `grasp_box` method! To illustrate the gained degrees of freedom, we also impose a random bias (leading to different solutions in nullspace):

In [12]:
C.setJointState(qHome)
limits = C.getJointLimits()
for orientation in ['x', 'y', 'z']:
    for i in range(10):
        # setup the manipulation problem
        man = manip.ManipulationModelling(C)
        man.setup_inverse_kinematics()
        # ... with random bias in joint space
        qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
        man.bias(1., qBias, 1e0)
        # ... and general, non-centered box grasping
        man.grasp_box(1., gripper, box, palm, orientation, margin=.02)
        
        # solve
        pose, ret = man.solve()
        
        # if feasible, display
        if ret.feasible:
            C.setJointState(pose[0])
            C.view(True, f'grasp_box with orientation {orientation}\nret: {ret}')
        else:
            print('-- infeasible', i, orientation)

-- infeasible 2 x
-- infeasible 5 y


### Cylinder grasp

A cylinder (or capsule) can be grasped by ensuring the finger axis is normal to the cylinder's axis -- have a look at the `grasp_cylinder` method. Again, a demo with random bias to show the variety of grasps modelled that way:

In [13]:
C.setJointState(qHome)
limits = C.getJointLimits()
for i in range(10):
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
    man.bias(1., qBias, 1e0)
    man.grasp_cylinder(1., gripper, 'capsule', palm)
    
    # solve
    pose, ret = man.solve()
    
    # if feasible, display
    if ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_cylinder\nret: {ret}')
    else:
        print('-- infeasible', i, orientation)

-- infeasible 1 z
-- infeasible 5 z


## Sequential Manipulation Modelling

Sequential manipulation modelling is special, as in some phases the manipulated objects move with the manipulator. Internally, komo models this with a *mode switch* (where an object becomes attached to the manipulator with a stable (but optimizable) relative transform).

Using the ManipulationModelling class, the `setup_pick_and_place_waypoints` method creates a two-time-steps komo problem where the relative object-gripper position is constrained to be the same in the 1st and 2st step (as it is parameterized by a shared relative kinematic joint). The `grasp_box` method ensures that the solution *also* fulfils grasp constraints in the first time step; and the `place_box` method ensures that the solution *also* fulfils placement constraints in the second time step. The additional `target_relative_xy_position` is optional, so see placement to explicit xy-positions on the table. Have a look at the definitions of all these methods.

In [14]:
C.setJointState(qHome)

for i in range(10):
    grasp_ori = random.choice(['x', 'y', 'z'])
    place_ori = 'z' #random.choice(['x', 'y', 'z'])
    info = f'pnp {i}, grasp orientation {grasp_ori}, place orientation {place_ori}'
    print('===', info)
    
    # setup manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_pick_and_place_waypoints(gripper, box)
    man.grasp_box(1., gripper, box, palm, grasp_ori)
    man.place_box(2., box, table, palm, place_ori)
    man.target_relative_xy_position(2., box, table, [(i%5)*.1-.2, .2])
    
    # solve
    q, ret = man.solve()

    # if feasible, display (including 'fake' simulation with kinematic attach)
    if ret.feasible:
        C.setJointState(q[0])
        C.view(True, f'{info}\nwaypoint 0\nret: {ret}')
        C.attach(gripper, box)
        C.setJointState(q[1])
        C.view(True, f'{info}\nwaypoint 1\nret: {ret}')
        C.attach(table, box)
        C.setJointState(qHome)
        C.view(True, 'back home')
    else:
        print(' -- infeasible')

del man
C.frame('box').setPosition([-.25,.1,1.])
C.view()

=== pnp 0, grasp orientation y, place orientation z
=== pnp 1, grasp orientation y, place orientation z
=== pnp 2, grasp orientation z, place orientation z
 -- infeasible
=== pnp 3, grasp orientation z, place orientation z
 -- infeasible
=== pnp 4, grasp orientation y, place orientation z
=== pnp 5, grasp orientation x, place orientation z
=== pnp 6, grasp orientation z, place orientation z
 -- infeasible
=== pnp 7, grasp orientation y, place orientation z
=== pnp 8, grasp orientation z, place orientation z
 -- infeasible
=== pnp 9, grasp orientation z, place orientation z
 -- infeasible


13

## Path generation

Once solutions to the manipulation keyframes/waypoints are available, the next step is to generate motion between them. We can use sample-based path finding (bi-directional RRT) and/or smooth motion optimization for this.

### Smooth point-to-point motion
The following demonstrates smooth point-to-point motion between box grasps, there the motion is additionally constrains the endeffector to retract and approach:

In [18]:
C.setJointState(qHome)
limits = C.getJointLimits()
verbose = 0

for i in range(20):
    qStart = C.getJointState()
    
    # choose a random grasp orientation
    orientation = random.choice(['x', 'y', 'z'])
    print('===', i, 'orientation', orientation)
    
    # setup the grasp problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics(accumulated_collisions=True)
    man.grasp_box(1., gripper, box, palm, orientation)
    
    # solve
    pose, ret = man.solve()
    print('    IK:', ret)
    
    # if feasible, display; otherwise try another grasp
    if ret.feasible:
        if verbose>0:
            C.setJointState(pose[0])
            C.view(True, f'grasp {i} with orientation {orientation}\nret: {ret}')
    else:
        print('  -- infeasible')
        C.setJointState(qStart)
        continue

    # setup the motion problem
    man = manip.ManipulationModelling(C, helpers=[gripper])
    man.setup_point_to_point_motion(qStart, pose[0])
    man.retract([.0, .2], gripper)
    man.approach([.8, 1.], gripper)
    
    # solve
    path, ret = man.solve()
    print('  path:', ret)

    # if feasible, display trivially (no real execution in BotOp here)
    if ret.feasible:
        for t in range(path.shape[0]):
            C.setJointState(path[t])
            C.view(False, f'grasp {i} with orientation {orientation}, path step {t}\n{ret}')
            time.sleep(.05)
        C.view(verbose>0, f'path done')
    else:
        print('  -- infeasible')
        
del man

=== 0 orientation y
    IK: { time: 0.037198, evals: 63, done: 1, feasible: 1, sos: 0.0097711, f: 0, ineq: 0, eq: 0.000842565 }
  path: { time: 0.071505, evals: 25, done: 1, feasible: 1, sos: 0.702696, f: 0, ineq: 8.06484e-07, eq: 0.000115799 }
=== 1 orientation z
    IK: { time: 0.045288, evals: 45, done: 1, feasible: 1, sos: 0.0236689, f: 0, ineq: 0, eq: 2.24476e-05 }
  path: { time: 0.066666, evals: 30, done: 1, feasible: 1, sos: 1.2123, f: 0, ineq: 1.51354e-06, eq: 0.00064524 }
=== 2 orientation x
    IK: { time: 0.051579, evals: 28, done: 1, feasible: 1, sos: 0.047128, f: 0, ineq: 8.41196e-09, eq: 7.75037e-07 }
  path: { time: 0.048498, evals: 23, done: 1, feasible: 1, sos: 1.82956, f: 0, ineq: 7.69909e-07, eq: 0.00041442 }
=== 3 orientation x
    IK: { time: 0.049027, evals: 32, done: 1, feasible: 1, sos: 0.047128, f: 0, ineq: 3.09808e-08, eq: 1.06089e-06 }
  path: { time: 0.063301, evals: 37, done: 1, feasible: 1, sos: 0.463497, f: 0, ineq: 1.28243e-06, eq: 0.000447379 }
=== 4 o

## Integrated Example

Let's start with an integrated example, where the robot endlessly loops through picking and placing a box on a table.

In [7]:
from importlib import reload
reload(manip)

<module 'manipulation' from '/home/mtoussai/git/rai-tutorials/manipulation.py'>

In [1]:
import robotic as ry
import manipulation as manip
import numpy as np
from importlib import reload
import time
import random

We define a basic configuration with box on the table:

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

C.addFrame("obstacle", "table") \
    .setShape(ry.ST.ssBox, [.06,.15,.06,.005]) \
    .setColor([.1]) \
    .setRelativePosition([-.15,.3-.055,.095]) \
    .setContact(1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

### endless box pick and place with random pick and place orientations

In [5]:
reload(manip)

C.setJointState(qHome)
C.view_raise()

for l in range(20):
        qStart = C.getJointState()

        graspDirection = random.choice(['y', 'z']) #'x' not possible: box too large
        placeDirection = random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C, info, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.no_collision([], palm, table)
        M.target_relative_xy_position(2., box, table, [.2, .3])
        ways = M.solve()

        if not M.feasible:
            continue

        M2 = M.sub_motion(0)
        # M = manip.ManipulationModelling(C, info, helpers=[gripper])
        # M.setup_point_to_point_motion(qStart, ways[0])
        M2.no_collision([.3,.7], palm, box, margin=.05)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.feasible:
            continue

        M3 = M.sub_motion(1)
        #manip.ManipulationModelling(C, info)
        # M.setup_point_to_point_motion(ways[0], ways[1])
        M3.no_collision([], table, box)
        M3.solve()
        if not M3.ret.feasible:
            continue

        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp z place zNeg
  -- infeasible:placement 0: grasp z place zNeg
     { time: 0.256131, evals: 200, done: 1, feasible: 0, sos: 0.365473, f: 0, ineq: 0.200974, eq: 1.08181 }
=== placement 1: grasp y place zNeg
  -- infeasible:placement 1: grasp y place zNeg
     { time: 0.217418, evals: 201, done: 1, feasible: 0, sos: 0.531033, f: 0, ineq: 0.232817, eq: 1.21976 }
=== placement 2: grasp y place xNeg
  -- infeasible:placement 2: grasp y place xNeg
     { time: 0.214746, evals: 200, done: 1, feasible: 0, sos: 0.367915, f: 0, ineq: 0.235169, eq: 1.12234 }
=== placement 3: grasp y place xNeg
  -- infeasible:placement 3: grasp y place xNeg
     { time: 0.207767, evals: 200, done: 1, feasible: 0, sos: 0.367921, f: 0, ineq: 0.235219, eq: 1.12266 }
=== placement 4: grasp y place y
  -- infeasible:placement 4: grasp y place y
     { time: 0.192758, evals: 200, done: 1, feasible: 0, sos: 0.248222, f: 0, ineq: 0.252934, eq: 2.54787 }
=== placement 5: grasp y place yNeg
  -- infea

### box top grasps and place over obstacle

In [17]:
#reload(manip)

C.setJointState(qHome)
C.view_raise()

C.frame(box).setRelativePosition([-.0,.3-.055,.095])
C.frame(box).setRelativeQuaternion([1.,0,0,0])

for i in range(7):
        qStart = C.getJointState()

        graspDirection = 'yz' #random.choice(['xz', 'yz'])
        placeDirection = 'z'
        place_position = [(i%3)*.3-.3, .2]
        place_orientation = [-(i%2),((i+1)%2),0.]
        info = f'placement {i}: grasp {graspDirection} place {placeDirection} place_pos {place_position} place_ori {place_orientation}'
        print('===', info)

        M = manip.ManipulationModelling(C, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, joint_limits=False)
        M.grasp_top_box(1., gripper, box, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, place_position)
        M.target_x_orientation(2., box, place_orientation)
        M.solve()
        if not M.feasible:
                continue

        M2 = M.sub_motion(0)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.ret.feasible:
            continue

        M3 = M.sub_motion(1)
#         M3.retract([.0, .2], box, distance=.05)
#         M3.approach([.8, 1.], box, distance=.05)
        M3.no_collision([], table, box)
        M3.no_collision([], box, 'obstacle')
        M3.bias(.5, qHome, 1e0)
        M3.solve()
        if not M3.ret.feasible:
            continue
            
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp yz place z place_pos [-0.3, 0.2] place_ori [0, 1, 0.0]
  -- feasible:
     { time: 0.019295, evals: 17, done: 1, feasible: 1, sos: 0.0717996, f: 0, ineq: 0, eq: 2.66157e-05 }
  -- feasible:sub_motion_0--
     { time: 0.057176, evals: 33, done: 1, feasible: 1, sos: 0.990973, f: 0, ineq: 5.84115e-07, eq: 6.16507e-05 }
  -- feasible:sub_motion_1--
     { time: 0.027784, evals: 9, done: 1, feasible: 1, sos: 1.49209, f: 0, ineq: 0.000491425, eq: 0.00364284 }
=== placement 1: grasp yz place z place_pos [0.0, 0.2] place_ori [-1, 0, 0.0]
  -- feasible:
     { time: 0.181115, evals: 201, done: 1, feasible: 1, sos: 0.24, f: 0, ineq: 0, eq: 0.131185 }
  -- feasible:sub_motion_0--
     { time: 0.039225, evals: 29, done: 1, feasible: 1, sos: 0.963553, f: 0, ineq: 2.5407e-06, eq: 0.000452594 }
  -- feasible:sub_motion_1--
     { time: 0.073592, evals: 35, done: 1, feasible: 1, sos: 5.71345, f: 0, ineq: 0.00208434, eq: 0.00855669 }
=== placement 2: grasp yz place z place_pos [0

In [13]:
reload(manip)

C.frame("l_panda_finger_joint1").setJointState(np.array([.0]))

obj = box
C.frame(obj).setRelativePosition([-.0,.3-.055,.095])
C.frame(obj).setRelativeQuaternion([1.,0,0,0])

for i in range(50):
     qStart = C.getJointState()

     info = f'push'
     M = manip.ManipulationModelling(C, info, ['l_gripper'])
     M.setup_pick_and_place_waypoints(gripper, obj, 1e-1)
     M.straight_push([1.,2.], obj, gripper, table)
     #random target position
     M.komo.addObjective([2.], ry.FS.position, [obj], ry.OT.eq, 1e1*np.array([[1,0,0],[0,1,0]]), .4*np.random.rand(3) - .2+np.array([.0,.3,.0]))
     M.solve()
     if not M.ret.feasible:
          continue

     M1 = M.sub_motion(0)
     M1.retractPush([.0, .15], gripper, .03)
     M1.approachPush([.85, 1.], gripper, .03)
     M1.no_collision([.15,.85], obj, "l_finger1", .02)
     M1.no_collision([.15,.85], obj, "l_finger2", .02)
     M1.no_collision([.15,.85], obj, 'l_palm', .02)
     M1.no_collision([], table, "l_finger1", .0)
     M1.no_collision([], table, "l_finger2", .0)
     M1.solve()
     if not M1.ret.feasible:
          continue

     M2 = M.sub_motion(1)
     M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
     M2.solve()
     if not M2.ret.feasible:
          continue

     M1.play(C, 1.)
     C.attach(gripper, obj)
     M2.play(C, 1.)
     C.attach(table, obj)


  -- feasible:push
     { time: 0.063781, evals: 60, done: 1, feasible: 1, sos: 0.1724, f: 0, ineq: 0.000526296, eq: 0.0492543 }
  -- feasible:sub_motion_0--push
     { time: 0.260757, evals: 58, done: 1, feasible: 1, sos: 2.06967, f: 0, ineq: 1.90198e-07, eq: 8.46764e-05 }
  -- feasible:sub_motion_1--push
     { time: 0.032384, evals: 9, done: 1, feasible: 1, sos: 0.23637, f: 0, ineq: 0, eq: 0.0005963 }
  -- infeasible:push
     { time: 0.205033, evals: 201, done: 1, feasible: 0, sos: 0.535202, f: 0, ineq: 0.418576, eq: 3.62175 }
  -- feasible:push
     { time: 0.019408, evals: 24, done: 1, feasible: 1, sos: 0.125938, f: 0, ineq: 0, eq: 0.0152847 }
  -- feasible:sub_motion_0--push
     { time: 0.35318, evals: 112, done: 1, feasible: 1, sos: 1.36037, f: 0, ineq: 1.68435e-05, eq: 0.0001211 }
  -- feasible:sub_motion_1--push
     { time: 0.020295, evals: 9, done: 1, feasible: 1, sos: 0.0802382, f: 0, ineq: 0, eq: 0.00174903 }
  -- infeasible:push
     { time: 0.091958, evals: 61, done: 1

## TODOS:
* Proper execution: BotOp instead of display with C
* RRTs
* additional planar motion constraint for in-plane manipulation
* more typical manipulation constraints: camera_look_at, push_straight, 

In [8]:
np.random.rand(3)

array([0.3025642 , 0.018585  , 0.80866885])

In [4]:
del C